<a href="https://colab.research.google.com/github/wwwh0225/elements-of-statistical-learning/blob/main/Ch7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model_Assessment_and_Selection 

在訓練模型時，我們除了單純在樣本內進行預測之外，我們也會希望建立出一個更一般化 **(generalized)** 的模型，在不同的情境之下皆能夠去使用。也因此，我們會想要建立一個評判準則來決定模型的優劣與品質。

## 模型的誤差
讓我們進到一個預測模型 $\hat{f(X)}$ ，我們希望透過這個模型去預測 $Y$ ，而一般來說，$\hat{f(X)}$ 是由訓練集 $\mathcal{T}$ 所估計而成。為了要衡量 $Y$ 與 $\hat{f(X)}$ 的差距，我通常會定義以下兩類的損失函數 (loss function)，以 $L(Y,\hat{f(X)})$ 表示：

$$L(Y,\hat{f(X)})=\left\{\begin{matrix}
 (Y-\hat{f(X)})^2  & 平方誤差 \\ 
 \left | Y-\hat{f(X)} \right | &  絕對值誤差
\end{matrix}\right.$$

現在，讓我們定義所謂**測試誤差 (test error)**，也就是來自於獨立的測試集樣本的一般化誤差。而當我們確立訓練集後，測試集也自動被確定。

- **測試誤差** (給定特定訓練集)：

$$\text{Err}_{\mathcal{T}} =E[L(Y,\hat{f(X)})|\mathcal{T}]$$

當然，給定「特定」訓練集時，我們能去對此測試誤差進行點估計，但是正因為「特定」訓練集也固定住了測試誤差，使得我們無法透過統計方法進行推論。

若我們將生成訓練集的隨機性一併考慮，也就是可以再對上式再取一個期望值，透過雙重期望值法則，我們可以得到**期望測試誤差 (expected test error** or **expected prediction error)**：

$$\text{Err}=E[L(Y,\hat{f(X)})] = E[E(L(Y,\hat{f(X)})|\mathcal{T})] = E[\text{Err}_{\mathcal{T}}]$$

一般而言，在訓練模型時，我們都希望透過某種方式去極小化訓練集的「損失」，因此我可們就定義**訓練誤差 (training error)** 如下：

- **訓練誤差** (訓練集的平均損失)：

$$\overline{\text{err}} = \frac{1}{N}\sum_{i=1}^N L(y_i,\hat{f(x_i)})$$

值得注意的一點是： $\overline{\text{err}}$ 是來自於已抽出的訓練集，因此在計算時，我們是使用實現值 $\{(x_i,y_i)\}_{i=1}^N$ 。

接著，我們就可以看到測試誤差於訓練集與測試及所扮演的角色。

<img src="https://esl.hohoweiya.xyz/img/07/fig7.1.png" alt="drawing" width="500"/>

由上圖可見，透過增加模型的複雜度，我們可以一直下降「訓練誤差」，這是來自於模型複雜度提升時，模型的**偏誤 (bias)** 會下降，但**變異 (variance)** 會取而代之地提升。因此，一味地追求訓練誤差下降是不切實際的，我們應該一併考慮「測試誤差」，在增加模型複雜度的同時，也要關心測試誤差是否反而提升。

*(通常我們會想找測試誤差最低點之處作為最終模型複雜度的選擇)*

當然，對於類別變數的 $Y$ 而言，我們同樣可以定義其損失函數，我們假設 $Y$ 的類別有 $K$ 類，而我們定義 $\hat{G(X)}= \mathop{\arg\max}\limits_k \hat{P(G=k|X)}$ 藉此將機率投射到類別之中，以下是常用的損失函數：

- $L(G,\hat{G(X)}) = I(G \neq \hat{G(X)})$ , (0-1 loss)

- $\begin{aligned}L(G,\hat{G(X)}) &= -2 \sum_{k=1}^K I(G=k) \ln \hat{p_k}(X) \\ &= -2 \ln \hat{p_G}(X)\end{aligned}$, ($-2 \times \text{log-likelihood}$)

---
**NOTE**

第二條損失函數，也就是在統計學習中常說的 **deviance**，詳細推導之過程及原因可見此篇[文章](https://convegni.unica.it/cladag2015/files/2015/10/Romeo1.pdf)。

---

我們同樣可以類推在分類問題中，**測試誤差** (或稱**錯誤分類誤差**) $\text{Err}_{\mathcal{T}} =E[L(G,\hat{G(X)})|\mathcal{T}]$。若對 $\text{Err}_{\mathcal{T}}$ 再取期望值，同樣可得**期望預測誤差** $\text{Err} = E[L(G,\hat{G(X)})]$。

對於分類問題的**訓練誤差**定義如下 (以 smaple log-likelihood 為例)：

$$\overline{\text{err}} = -\frac{2}{N}\sum_{i=1}^N \ln  \hat{p_{g_i}}(x_i)$$

在此，有兩個名詞需要被清楚定義：

1. **模型選擇 Model selection**：估計不同模型的表現，並藉此找到最好的模型
2. **模型評估 Model assessment**：當選擇完最終模型後，透過獨立樣本資料來計算預測誤差 $\text{err}$


##  偏誤與變異的權衡 The bias-variance tradeoff

一般而言，在統計模型的架構下，我們假設模型有這樣的關係：

$$Y = f(X) + \varepsilon $$

其中

$$E(\varepsilon )=0, \ Var(\varepsilon)=\sigma_{\varepsilon }^2$$

以均方差的損失函數為例，模型 $\hat{f}(X=x_0)$ 的誤差 (error) 可以拆寫成下列形式：








![](https://latex.codecogs.com/gif.latex?%5Cbegin%7Baligned%7D%5Ctext%7BErr%7D%28x%29%26%3DE%5B%28%7B%5Ccolor%7BRed%7D%20Y%7D-%5Chat%7Bf%7D%28X%29%29%5E2%7CX%3Dx%5D%20%5C%5C%26%3DE%5B%7B%5Ccolor%7BRed%7D%20%28f%28X%29%20&plus;%20%5Cvarepsilon%7D-%5Chat%7Bf%7D%28X%29%29%5E2%7CX%3Dx%5D%20%5C%5C%26%3D%5Csigma_%5Cvarepsilon%5E2%20&plus;E%5B%28f%28x%29-%5Chat%7Bf%7D%28x%29%29%5E2%5D%20&plus;%20%7B%5Ccolor%7BBlue%7D%20%5Cunderbrace%7B2E%5B%5Cvarepsilon%20%28f%28x%29-%5Chat%7Bf%7D%28x%29%29%5D%7D_0%7D%20%5C%5C%26%3D%20%5Csigma_%5Cvarepsilon%5E2%20&plus;E%5B%28f%28x%29%20%7B%5Ccolor%7BRed%7D%20-E%5Chat%7Bf%7D%28x%29%20&plus;E%5Chat%7Bf%7D%28x%29%7D%20-%5Chat%7Bf%7D%28x%29%29%5E2%5D%20%5C%5C%26%3D%20%5Csigma_%5Cvarepsilon%5E2&plus;%20E%5B%28f%28x%29%20-E%5Chat%7Bf%7D%28x%29%29%5E2%5D%20&plus;%20E%5B%28E%5Chat%7Bf%7D%28x%29%20-%5Chat%7Bf%7D%28x%29%29%5E2%5D%20&plus;%20%7B%5Ccolor%7BBlue%7D%20%5Cunderbrace%7B2%20E%5B%28f%28x%29%20-E%5Chat%7Bf%7D%28x%29%29%5D%5B%28E%5Chat%7Bf%7D%28x%29%20-%5Chat%7Bf%7D%28x%29%29%5D%7D_0%7D%20%5C%5C%26%3D%20%7B%5Ccolor%7BBrown%7D%20%5Csigma_%5Cvarepsilon%5E2%20%7D%20&plus;%20%7B%5Ccolor%7BOrange%7D%20%5BE%5Chat%7Bf%7D%28x%29%20-f%28x%29%5D%5E2%7D%20&plus;%20%7B%5Ccolor%7BOrchid%7D%20E%5B%28%5Chat%7Bf%7D%28x%29-E%5Chat%7Bf%7D%28x%29%29%5E2%5D%7D%20%5C%5C%26%3D%20%7B%5Ccolor%7BBrown%7D%20%5Ctext%7BIrreducible%20Error%7D%7D%20&plus;%20%7B%5Ccolor%7BOrange%7D%20%5Ctext%7BBias%7D%5E2%7D%20&plus;%20%7B%5Ccolor%7BOrchid%7D%20%5Ctext%7BVariance%7D%7D%5Cend%7Baligned%7D)

以最小平方法的 $p$ 維線性迴歸模型 ( $\hat{f_p}(x) = x^T\hat{\beta}$ ) 為例：

我們令 $h(x) = X(X^TX)^{-1}x$ ，因此 $\hat{f_p}(x) = x^T (X^TX)^{-1} X^T y = h^T(x) y$ 

則 

$$Var[\hat{f_p}(x)]=\left \|  h(x) \right \|^2 \sigma_\varepsilon ^2$$

因此

$$\begin{aligned} \text{Err} (x) &= E[(Y-\hat{f_p}(x))^2|X=x] \\ &= \sigma_\varepsilon ^2 + (f(x)-E[\hat{f_p}(x)])^2 + \left \|  h(x) \right \|^2 \sigma_\varepsilon ^2 \end{aligned}$$


我們可以接著計算所謂**樣本內 (in-sample) 誤差**：

$$\frac{1}{N}\sum_{i=1}^N \text{Err} (x_i) = \sigma_\varepsilon ^2 + \frac{1}{N}\sum_{i=1}^N (f(x_i)-E[\hat{f}(x_i)])^2 +\frac{p}{N} \sigma_\varepsilon ^2$$

---
**NOTE**

$$\begin{aligned}
\sum_{i=1}^N\left \|  h(x_i) \right \|^2 &=\sum_{i=1}^N h^T(x_i)h(x_i)\\
&= \sum_{i=1}^N x_i^T(X^TX)^{-1}x_i \\
&= trace[X(X^TX)^{-1}X^T] \\
&= trace[(X^TX)^{-1}X^TX] \\
&= p
\end{aligned}$$

---

<img src="https://esl.hohoweiya.xyz/img/07/fig7.2.png" alt="drawing" width="500"/>



圖 7.2 表現出模型與「真實」機率分佈之間的關係

---
<img src="https://esl.hohoweiya.xyz/img/07/fig7.3.png" alt="drawing" width="500"/>





## 樂觀的訓練誤差

正如先前所介紹的各式誤差 (errors)，誤差可以從樣本 (訓練集) 內與樣本外去衡量，在此我們關注由訓練集所計算而成的誤差。

我們已經定義過**「訓練誤差」**為：

$$\overline{\text{err}} = \frac{1}{N}\sum_{i=1}^N L(y_i,\hat{f(x_i)})$$

而**樣本內 (in-sample) 誤差**，就是「遍歷」過全部的樣本點 $x_i$ 並且考慮在這些點上 $Y$ 的隨機性： 

$$\text{Err}_{in}=\frac{1}{N}\sum_{i=1}^N E[L(Y,\hat{f(x_i)})|\mathcal{T}] $$

由於樣本內誤差 $\text{Err}_{in}$ 相對於訓練誤差 $\overline{\text{err}}$ 多考慮到 $Y$ 的隨機性，因此樣本內誤差所算出之值應會**「高於」**訓練誤差之值。換句話說，若使用訓練誤差作為模型好壞的衡量，我們可能會**「樂觀地低估」**實際的誤差，其中的差異，可以被定義為**「誤差的樂觀性 (optimism)」**，由於前述的原因，此樂觀性 $\text{op}$ 應在大多時候為一個正數。

$$\text{op} = \text{Err}_{in} - \overline{\text{err}}$$


接著，我們可以定義**「平均樂觀 $\omega$」**，為針對 $\text{op}$ 再取期望值，我們可以透過重複抽樣的方式對其進行估計。

$$\omega = E_y[\text{op}]$$

若給定損失函數為 squared-loss，則

$$\omega = E_y[\text{op}]= \frac{2}{N} \sum_{i=1}^N Cov(\hat{y_i},y_i)$$

證明如下：

---
<img src="https://user-images.githubusercontent.com/13688320/51331586-095cf300-1ab5-11e9-9a21-b9ec2902d9ec.png" alt="drawing" width="500"/>
<img src="https://user-images.githubusercontent.com/13688320/51331590-0b26b680-1ab5-11e9-8c7e-bb83000f67f5.png" alt="drawing" width="500"/>

首先，我們先令

$$\hat{y_i} = \hat{f}(x_i)$$

並且了解到 $E_{Y^0}Y_i^0 = E_y y_i$，這是由於不論是 $Y_i^0$ 或 $y_i$ 皆可以被視為從等同分配中所抽出。

<!--算式圖片檔 https://upload.cc/i1/2022/11/07/W1QVky.gif  -->

$$\begin{aligned}
\omega &= E_y[\text{op}] = E_y[\text{Err}_{in} - \overline{\text{err}}] \\
&=E_y[\frac{1}{N}\sum_{i=1}^N E_{Y^0}(Y_i^0-\hat{f}(x_i))^2-\frac{1}{N}\sum_{i=1}^N (y_i-\hat{f}(x_i))^2] \\
&= \frac{1}{N}\sum_{i=1}^N\{  E_yE_{Y^0}(Y_i^0 - \hat{y_i})^2-E_y(y_i-\hat{y_i})^2  \}\\
&= \frac{1}{N}\sum_{i=1}^N\{  E_yE_{Y^0}[(Y_i^0)^2-2Y_i^0\hat{y_i}+\hat{y_i}^2]-E_y(y_i^2-2y_i\hat{y_i}+\hat{y_i}^2)  \}\\
&= \frac{1}{N}\sum_{i=1}^N \{{\color{Blue} E_{Y^0}(Y_i^0)^2}  -2E_yE_{Y^0}Y_i^0\hat{y_i} {\color{Red} +E_y\hat{y_i}^2} {\color{Blue} -E_y y_i^2}+2E_y y_i\hat{y_i}{\color{Red} -E_y\hat{y_i}}  \} \\
&= \frac{1}{N}\sum_{i=1}^N\{ 2E_y y_i\hat{y_i} -2E_yE_{Y^0}Y_i^0\hat{y_i}  \}\\
&=\frac{1}{N}\sum_{i=1}^N\{ 2E_y y_i\hat{y_i} -2{\color{Teal} E_{Y^0}Y_i^0}E_y\hat{y_i}  \}\\
&=\frac{1}{N}\sum_{i=1}^N\{ 2E_y y_i\hat{y_i} -2{\color{Teal} E_y y_i}E_y\hat{y_i}  \} \\
&= \frac{2}{N}\sum_{i=1}^N\{E_y y_i\hat{y_i} - E_y y_iE_y\hat{y_i} \}\\
&= \frac{2}{N}\sum_{i=1}^N Cov(\hat{y_i},y_i)
\end{aligned}$$

---

已知 $Y = f(X) + \varepsilon$，若 $\hat{y_i}$ 是由 $d$ 個變數 (inputs) 或是 basis functions 所計算而成，則

$$\sum_{i=1}^NCov(\hat{y_i},y_i) = d \sigma^2_\varepsilon$$

一個廣義的證明如下：

<img src="https://user-images.githubusercontent.com/13688320/61992768-ff840000-b094-11e9-9cdd-ddb50c3bfd14.png" alt="drawing" width="500"/>

知 $Y = f(X) + \varepsilon$ 且 $E(\varepsilon)=0$、$Var(\varepsilon)=\sigma^2_\varepsilon$

且定義 $s_i$ 為 $S$ 的第 $i$ 個 row、$S_{ii}$ 為 $S$ 的第 $i$ 個對角線元素。

則

$$\begin{aligned}
\sum_{i=1}^NCov(\hat{y_i},y_i) &= \sum_{i=1}^NCov(s^T_i \mathbf{y},y_i) = \sum_{i=1}^NCov(S_{ii} y_i,y_i) \\
&=\sum_{i=1}^N S_{ii}Cov( y_i,y_i) = \sum_{i=1}^N S_{ii}Var(y_i) \\
&=\sum_{i=1}^N S_{ii} \sigma^2_\varepsilon = \sigma^2_\varepsilon \sum_{i=1}^N S_{ii} \\
&=\sigma^2_\varepsilon trace(S)
\end{aligned}$$

會得到這個結果，是由於 $\hat{y_i}$ 是 $\mathbf{y}$ 的一個線性組合，而 $\mathbf{y} = (y_1,y_2,\cdots,y_N)^T$ ，由於我們的資料來自獨立隨機樣本，故 $Cov(y_i,y_j)=0, \forall i \neq j$，故我們只需在意 $s_i^T$ 這個向量裡面的第 $i$ 個元素即可，由於 $S$ 是一個 $N \times N$ 的方陣，故這個元素正好是第 $i$ 個對角線元素。因此，將矩陣中之 $N$ 個對角線元素相加即為 $trace$ 的定義，故得證。

---

對於樣本內誤差的估計，一般而言有三個經典的估計方法，分別為 Mallow's Cp、AIC 以及 BIC。

- Cp

- AIC

- BIC

https://www.stat.cmu.edu/~larry/=stat401/lecture-21.pdf

---




## 交叉驗證 Cross Validation

### K-fold Cross Validation


### 正確地使用交叉驗證
